<a href="https://colab.research.google.com/github/cherrka/tamilla-latypova/blob/main/%D0%A3%D1%87%D0%B5%D1%82%20%D0%B8%20%D1%83%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%82%D1%80%D0%B0%D0%BD%D1%81%D0%BF%D0%BE%D1%80%D1%82%D0%BD%D0%BE%D0%B9%20%D0%BA%D0%BE%D0%BC%D0%BF%D0%B0%D0%BD%D0%B8%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import json
import logging
from typing import Optional, Dict, List
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[logging.FileHandler('transport_company.log'), logging.StreamHandler()])
class InvalidVehicleError(Exception):
    """Исключение для некорректных данных транспортного средства."""
    pass
class PermissionDeniedError(Exception):
    """Исключение для отсутствия прав доступа."""
    pass
class DriverNotFoundError(Exception):
    """Исключение для отсутствия водителя."""
    pass
class VehicleMeta(type):
    registry: Dict[str, 'Vehicle'] = {}

    def __init__(cls, name, bases, attrs):
        super().__init__(name, bases, attrs)
        if name != 'Vehicle':
            VehicleMeta.registry[name.lower()] = cls
class Vehicle(metaclass=VehicleMeta):
    def __init__(self, vehicle_id: str, model: str, year: int, capacity: int, status: str):
        self.__vehicle_id = vehicle_id
        self.__model = model
        self.__year = year
        self.__capacity = capacity
        self.__status = status
    def calculate_cost(self) -> float:
        raise NotImplementedError("Этот метод должен быть переопределен в подклассах")
    def to_dict(self) -> dict:
        """Преобразует объект в словарь."""
        return {
            'vehicle_id': self.__vehicle_id,
            'model': self.__model,
            'year': self.__year,
            'capacity': self.__capacity,
            'status': self.__status
        }
    @classmethod
    def from_dict(cls, data: dict) -> 'Vehicle':
        """Создает объект из словаря."""
        return cls(data['vehicle_id'], data['model'], data['year'], data['capacity'], data['status'])

    def __str__(self) -> str:
        return f"Транспортное средство: {self.__model}, Год выпуска: {self.__year}"

    def __lt__(self, other: 'Vehicle') -> bool:
        return self.__year < other.__year

    def __gt__(self, other: 'Vehicle') -> bool:
        return self.__year > other.__year

    def __eq__(self, other: 'Vehicle') -> bool:
        return self.__vehicle_id == other.__vehicle_id

    @property
    def vehicle_id(self) -> str:
        return self.__vehicle_id

    @property
    def model(self) -> str:
        return self.__model

    @property
    def year(self) -> int:
        return self.__year

    @property
    def capacity(self) -> int:
        return self.__capacity

    @property
    def status(self) -> str:
        return self.__status

    @status.setter
    def status(self, value: str) -> None:
        self.__status = value
class Bus(Vehicle):
    def __init__(self, vehicle_id: str, model: str, year: int, capacity: int, route_number: str, status: str):
        super().__init__(vehicle_id, model, year, capacity, status)
        self.__route_number = route_number

    def calculate_cost(self) -> float:
        cost_calculator = BusCostCalculator()
        return cost_calculator.calculate_cost()

    def to_dict(self) -> dict:
        data = super().to_dict()
        data['route_number'] = self.__route_number
        return data

    @classmethod
    def from_dict(cls, data: dict) -> 'Bus':
        vehicle = super().from_dict(data)
        return cls(vehicle.vehicle_id, vehicle.model, vehicle.year, vehicle.capacity, data['route_number'], vehicle.status)

    def __str__(self) -> str:
        return f"Автобус: {self.model}, Номер маршрута: {self.__route_number}"

class Truck(Vehicle):
      def __init__(self, vehicle_id: str, model: str, year: int, capacity: int, cargo_capacity: float, status: str):
        super().__init__(vehicle_id, model, year, capacity, status)
        self.__cargo_capacity = cargo_capacity

      def calculate_cost(self) -> float:
        cost_calculator = TruckCostCalculator()
        return cost_calculator.calculate_cost()

      def to_dict(self) -> dict:
        data = super().to_dict()
        data['cargo_capacity'] = self.__cargo_capacity
        return data

      @classmethod
      def from_dict(cls, data: dict) -> 'Truck':
        vehicle = super().from_dict(data)
        return cls(vehicle.vehicle_id, vehicle.model, vehicle.year, vehicle.capacity, data['cargo_capacity'], vehicle.status)

      def __str__(self) -> str:
        return f"Грузовик: {self.model}, Грузоподъемность: {self.__cargo_capacity} тонн"


class Taxi(Vehicle):
    def __init__(self, vehicle_id: str, model: str, year: int, capacity: int, license_plate: str, status: str):
        super().__init__(vehicle_id, model, year, capacity, status)
        self.__license_plate = license_plate

    def calculate_cost(self) -> float:
        cost_calculator = TaxiCostCalculator()
        return cost_calculator.calculate_cost()

    def to_dict(self) -> dict:
        data = super().to_dict()
        data['license_plate'] = self.__license_plate
        return data

    @classmethod
    def from_dict(cls, data: dict) -> 'Taxi':
        vehicle = super().from_dict(data)
        return cls(vehicle.vehicle_id, vehicle.model, vehicle.year, vehicle.capacity, data['license_plate'], vehicle.status)

    def __str__(self) -> str:
        return f"Такси: {self.model}, Номерной знак: {self.__license_plate}"


class Address:
    def __init__(self, street: str, city: str):
        self.street = street
        self.city = city

    def __str__(self) -> str:
        return f"{self.street}, {self.city}"


class Driver:
    def __init__(self, name: str, driver_id: str, license_type: str, address: Address):
        self.name = name
        self.driver_id = driver_id
        self.license_type = license_type
        self.assigned_vehicle: Optional[Vehicle] = None
        self.address = address

    def assign_vehicle(self, vehicle: Vehicle) -> None:
        """Назначает транспортное средство водителю."""
        if not isinstance(vehicle, Vehicle):
            raise InvalidVehicleError("Некорректные данные транспортного средства.")

        logging.info(f"Водитель {self.name} назначен на транспортное средство {vehicle.model}.")
        self.assigned_vehicle = vehicle

    def remove_vehicle(self) -> None:
        """Удаляет назначенное транспортное средство у водителя."""
        logging.info(f"Транспортное средство {self.assigned_vehicle.model} удалено у водителя {self.name}.")
        self.assigned_vehicle = None

    def get_assigned_vehicle(self) -> Optional[Vehicle]:
        """Возвращает назначенное транспортное средство."""
        return self.assigned_vehicle


class Trackable:
    def track_location(self) -> str:
        pass


class Reportable:
    def generate_report(self) -> str:
        pass


class LoggingMixin:
    def log_action(self, action: str) -> None:
        logging.info(f"Логирование действия: {action}")


class NotificationMixin:
    def send_notification(self, message: str) -> None:
        logging.info(f"Уведомление: {message}")


class BusWithMixins(Bus, LoggingMixin, NotificationMixin):
    def calculate_cost(self) -> float:
        cost = super().calculate_cost()
        self.log_action(f"Расчет стоимости автобуса {self.model}: {cost}")
        return cost


class TruckWithMixins(Truck, LoggingMixin, NotificationMixin):
    def calculate_cost(self) -> float:
        cost = super().calculate_cost()
        self.log_action(f"Расчет стоимости грузовика {self.model}: {cost}")
        return cost
class TaxiWithMixins(Taxi, LoggingMixin, NotificationMixin):
    def calculate_cost(self) -> float:
        cost = super().calculate_cost()
        self.log_action(f"Расчет стоимости такси {self.model}: {cost}")
        return cost


class CostCalculator:
    """Базовый класс для расчета стоимости эксплуатации транспортных средств."""

    def calculate_cost(self) -> float:
        base_cost = self.get_base_cost()
        additional_costs = self.get_additional_costs()

        total_cost = base_cost + additional_costs
        logging.info(f"Общая стоимость эксплуатации: {total_cost}")

        return total_cost

    def get_base_cost(self) -> float:
        """Получает базовую стоимость эксплуатации. Должен быть переопределен в подклассах."""
        raise NotImplementedError("Этот метод должен быть переопределен в подклассах.")

    def get_additional_costs(self) -> float:
        """Получает дополнительные расходы. Может быть переопределен в подклассах."""
        return 0.0


class BusCostCalculator(CostCalculator):

    def get_base_cost(self) -> float:
        logging.info("Получение базовой стоимости для автобуса.")
        return 100.0


class TruckCostCalculator(CostCalculator):

    def get_base_cost(self) -> float:
        logging.info("Получение базовой стоимости для грузовика.")
        return 150.0


class TaxiCostCalculator(CostCalculator):

    def get_base_cost(self) -> float:
        logging.info("Получение базовой стоимости для такси.")
        return 80.0


def check_permissions(func):
    """Декоратор для проверки прав доступа перед выполнением функции."""

    def wrapper(*args, **kwargs):
        user_permissions = kwargs.get('user_permissions', [])

        if 'manage_vehicles' not in user_permissions:
            raise PermissionDeniedError("У вас нет прав доступа для выполнения этого действия.")

        return func(*args, **kwargs)

    return wrapper


class VehicleFactory:

    @staticmethod
    def create_vehicle(vehicle_type: str, *args) -> Vehicle:
        """Создает экземпляр транспортного средства по типу."""

        vehicle_class = VehicleMeta.registry.get(vehicle_type.lower())

        if not vehicle_class:
            raise ValueError(f"Тип транспортного средства '{vehicle_type}' не зарегистрирован.")

        return vehicle_class(*args)


class VehiclePark:

    def __init__(self):
        self.vehicles: List[Vehicle] = []

    @check_permissions
    def add_vehicle(self, vehicle: Vehicle, user_permissions=None):
        """Добавляет транспортное средство в парк."""

        if not isinstance(vehicle, Vehicle):
            raise InvalidVehicleError("Некорректные данные транспортного средства.")

        logging.info(f"Добавление транспортного средства {vehicle.model} в парк.")

        self.vehicles.append(vehicle)

    @check_permissions
    def remove_vehicle(self, vehicle_id: str, user_permissions=None):
        """Удаляет транспортное средство из парка по его идентификатору."""

        for vehicle in self.vehicles:
            if vehicle.vehicle_id == vehicle_id:
                logging.info(f"Удаление транспортного средства {vehicle.model} из парка.")
                self.vehicles.remove(vehicle)
                return

        raise InvalidVehicleError("Транспортное средство не найдено.")

    def get_all_vehicles(self) -> List[Vehicle]:
        """Возвращает список всех транспортных средств в парке."""

        logging.info("Получение списка всех транспортных средств в парке.")

        return self.vehicles

    def search_by_model(self, model: str) -> List[Vehicle]:
        """Ищет транспортные средства по модели."""
        found_vehicles = [vehicle for vehicle in self.vehicles if vehicle.model.lower() == model.lower()]

        logging.info(f"Найдено {len(found_vehicles)} транспортных средств по модели '{model}'.")

        return found_vehicles


class MaintenanceHandler:

    def set_next(self, handler: 'MaintenanceHandler') -> 'MaintenanceHandler':
        """Устанавливает следующий обработчик в цепочке."""

        self._next_handler = handler
        return handler

    def handle(self, request: float) -> None:
         """Обрабатывает запрос на техническое обслуживание."""

         if hasattr(self, '_next_handler'):
             self._next_handler.handle(request)


class Mechanic(MaintenanceHandler):

    def handle(self, request: float) -> None:
         """Обрабатывает запрос на техническое обслуживание от механика."""

         if request < 1000:
             logging.info("Механик одобрил обслуживание.")
         else:
             super().handle(request)


class DepartmentHead(MaintenanceHandler):

    def handle(self, request: float) -> None:
         """Обрабатывает запрос на техническое обслуживание от руководителя отдела."""

         if 1000 <= request < 5000:
             logging.info("Руководитель отдела одобрил обслуживание.")
         else:
             logging.warning("Обслуживание требует более высокого уровня одобрения.")


if __name__ == "__main__":

    driver_address = Address("123 Main St", "Cityville")

    driver = Driver("Иван Иванов", "D001", "B", driver_address)

    park = VehiclePark()

    bus_with_mixin = VehicleFactory.create_vehicle("bus", "B001", "Mercedes", 2020, 50, "10", "active")

    truck_with_mixin = VehicleFactory.create_vehicle("truck", "T001", "Volvo", 2019, 20, 5.0, "active")

    taxi_with_mixin = VehicleFactory.create_vehicle("taxi", "T002", "Toyota", 2021, 4, "ABC123", "active")

    user_permissions = ['manage_vehicles']

    try:
        park.add_vehicle(bus_with_mixin, user_permissions=user_permissions)
        park.add_vehicle(truck_with_mixin, user_permissions=user_permissions)
        park.add_vehicle(taxi_with_mixin, user_permissions=user_permissions)

        driver.assign_vehicle(bus_with_mixin)

        print(bus_with_mixin)
        bus_cost = bus_with_mixin.calculate_cost()
        print(f"Стоимость эксплуатации автобуса: {bus_cost}")

        mechanic = Mechanic()
        department_head = DepartmentHead()

        mechanic.set_next(department_head)

        maintenance_request_amounts = [500.0, 1500.0]

        for amount in maintenance_request_amounts:
            print(f"\nЗапрос на обслуживание на сумму {amount}:")
            mechanic.handle(amount)

        with open('vehicles.json', 'w') as file:
            json.dump([vehicle.to_dict() for vehicle in park.get_all_vehicles()], file)

        print("\nДанные о транспортных средствах сохранены в vehicles.json.")

        with open('vehicles.json', 'r') as file:
            vehicles_data = json.load(file)
            for data in vehicles_data:
                vehicle_class = VehicleMeta.registry.get(data['model'].lower())
                if vehicle_class is not None:
                    vehicle_instance = vehicle_class.from_dict(data)
                    park.add_vehicle(vehicle_instance, user_permissions=user_permissions)

        print("\nДанные о транспортных средствах загружены из vehicles.json.")

    except (InvalidVehicleError, PermissionDeniedError) as e:
        print(f"Ошибка: {e}")

Автобус: Mercedes, Номер маршрута: 10
Стоимость эксплуатации автобуса: 100.0

Запрос на обслуживание на сумму 500.0:

Запрос на обслуживание на сумму 1500.0:

Данные о транспортных средствах сохранены в vehicles.json.

Данные о транспортных средствах загружены из vehicles.json.
